In [1]:
# 0. Importar pacotes 

import pandas as pd
import numpy as np
import datetime as dt
import re
from ydata_profiling import ProfileReport

In [10]:
# 1. Importar base de dados  

base_155 = pd.read_excel(r'155_290825.XLSX')

In [11]:
# 1.1. Corrigindo tipo de campo e criando campo data_saida 
base_155['produto_retirar_dtpreventrega'] = pd.to_datetime(base_155['produto_retirar_dtpreventrega'], errors='coerce')
base_155['data_saida'] = base_155['produto_retirar_dtpreventrega'].dt.date 

C:\Users\lsaju\AppData\Local\Temp\ipykernel_32072\866468498.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  base_155['produto_retirar_dtpreventrega'] = pd.to_datetime(base_155['produto_retirar_dtpreventrega'], errors='coerce')


In [12]:
# 1.2. Criando campo dias_espera
#base_155_p2 = base_155[base_155['produto_retirar_idlocalretirada']==2].copy()
base_155_p2 = base_155.copy()
base_155_p2['dias_espera'] = pd.to_datetime(base_155_p2['data_saida'])
base_155_p2['data_saida']= pd.to_datetime(base_155_p2['data_saida'])
base_155_p2['produto_retirar_dtmovimento'] = pd.to_datetime(base_155_p2['produto_retirar_dtmovimento'])
base_155_p2['dias_espera'] = (base_155_p2['data_saida'] - base_155_p2['produto_retirar_dtmovimento']).dt.days
#base_155_p2.dtypes

In [ ]:
reportp2 = ProfileReport(base_155_p2, title='Base P2')
reportp2.to_notebook_iframe()

In [13]:
# 1.3. Descrição dos dados
#base_155_p2.describe()

In [19]:
# 1.4. Criação do campo do nome do vendedor
def extrair_primeiro_nome(texto):
    """
    Extrai apenas o PRIMEIRO nome, tratando diferentes padrões:
    - Nomes antes de números
    - Números antes de nomes  
    - Números colados com nomes
    - Múltiplos nomes
    """
    if pd.isna(texto) or texto.strip() == '':
        return ''
    
    texto = str(texto).strip()
    
    # Etapa 1: Remove caracteres especiais comuns em telefones
    # Mantém letras, espaços e números, remove (), -, etc.
    texto_limpo = re.sub(r'[^\w\s]', ' ', texto)
    texto_limpo = re.sub(r'\s+', ' ', texto_limpo).strip()
    
    # Etapa 2: Identifica todas as palavras que são apenas letras (nomes)
    palavras = texto_limpo.split()
    nomes_puros = []
    
    for palavra in palavras:
        # Se a palavra contém apenas letras (é um nome)
        if palavra.isalpha() and len(palavra) > 1:  # Ignora letras soltas
            nomes_puros.append(palavra)
    
    # Etapa 3: Se encontrou nomes, retorna apenas o primeiro
    if nomes_puros:
        return nomes_puros[0].upper()
    
    # Etapa 4: Se não encontrou nomes puros, tenta extrair de texto colado
    # Procura sequência de letras no início ou após números
    matches = re.findall(r'[A-Za-z]{2,}', texto)
    if matches:
        return matches[0].upper()
    
    # Se nada funcionou, retorna vazio
    return ''

# Aplica a função
base_155_p2['primeiro_nome'] = base_155_p2['cliente_fornecedor_nomevendedor'].apply(extrair_primeiro_nome)

print("\n✅ RESULTADO DA EXTRAÇÃO:")
print("=" * 80)
resultado = base_155_p2[['cliente_fornecedor_nomevendedor', 'primeiro_nome']].copy()
resultado.index = range(1, len(resultado) + 1)
#print(resultado.to_string())
base_155_p2.columns.tolist()



✅ RESULTADO DA EXTRAÇÃO:


['produto_retirar_dtmovimento',
 'produto_retirar_idlocalretirada',
 'produto_retirar_dtpreventrega',
 'local_retirada_descrlocalretirada',
 'produtos_view_idsubproduto',
 'produtos_view_descricaoproduto',
 'Quantidade Produto',
 'Número Nota',
 'Série Nota',
 'cliente_fornecedor_idclifor',
 'cliente_fornecedor_nome',
 'estoque_analitico_idvendedor',
 'cliente_fornecedor_nomevendedor',
 'data_saida',
 'dias_espera',
 'primeiro_nome']

In [18]:
# 1.5. Renomear nome das variáveis
base_155_semi = base_155_p2.copy()
base_155_semi.rename(columns={
    'produto_retirar_dtmovimento':'dtmovimento',
    'produto_retirar_idlocalretirada':'idlocalretirada',
    'produto_retirar_dtpreventrega':'dtpreventrega',
    'local_retirada_descrlocalretirada':'descrlocalretirada',
    'produtos_view_idsubproduto':'idsubproduto',
    'produtos_view_descricaoproduto':'descricaoproduto',
    'Quantidade Produto':'qtde_produto',
    'Numero Nota':'num_nota',
    'Série Nota':'serie_nota',
    'cliente_fornecedor_idclifor':'idclifor',
    'cliente_fornecedor_nome':'fornecedor_nome',
    'estoque_analitico_idvendedor':'idvendedor',
    'cliente_fornecedor_nomevendedor':'nomevendedor'
    },
    inplace=True)

base_155_semi.columns.tolist()

['dtmovimento',
 'idlocalretirada',
 'dtpreventrega',
 'descrlocalretirada',
 'idsubproduto',
 'descricaoproduto',
 'qtde_produto',
 'Número Nota',
 'serie_nota',
 'idclifor',
 'fornecedor_nome',
 'idvendedor',
 'nomevendedor',
 'data_saida',
 'dias_espera',
 'primeiro_nome']

In [17]:
# 2. Criar base de dados para exportação

base_155_semi.to_excel('base_155.xlsx')